# OpenStreetMap Data Case Study


## Map area
Norman, OK, United States
* Data Source: https://overpass-api.de/api/map?bbox=-97.7303,34.8358,-96.9942,35.6562
* Map: https://www.openstreetmap.org/relation/185049

I chose this area because it is my hometown. I am interested to see what facts I can pull from this data and I would like to contribute to the area's improvement on OpenStreetMap.org. Norman is also home to the University of Oklahoma, which I think may create some interesting details in the map data. The data file is 57.2 MB uncompressed.

# Problems Encountered in the Map

After running my initial audit scripts (audit_streetnames.py and audit_postcodes.py) I noticed three main problems with the data. 
* Inconsistently abbreviated street names ("SW 24th St.", "Southwest 24th Street")
* Incorrect zip codes ("c")
* Missing street name information ("Maple" instead of "Maple Avenue")

### Inconsistenly abbreviated street names
During the audit of the data, I noticed that there were some inconsistencies with how street names were abbreviated. Sometimes it would be St, sometimes St., event Steet a few times for Street. There were also differences in how directional information was stored. For example, N vs North. To fix these inconsistencies, I wrote code that split the street name into individual words and checked each one against my mapping to replace when needed. You will also note in the function that it handles 2 special cases. There were incomplete addresses and in one instance the zip code for the street name. I was able to find the correct information on the internet and used that as a direct replacement. I used the following code, with mapping omitted: 

```python
def rename_street_name(name):
    words = name.split()
    for w in range(len(words)):
        if(words[w] in mapping):
            if(words[w - 1].lower() not in ['suite', 'ste.', 'ste']):
                words[w] = mapping[words[w]]
    name = ' '.join(words)
    # special case for street name of just "Maple". I looked up the address
    if name.lower() == 'maple':
        name = 'Maple Avenue'
    # special case for street name of just "73069". I looked up the address
    if name == '73069':
        name = 'West Robinson Avenue'
    return name 
```


In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

### Postal Codes
There was similar problems with postal codes. Luckily they all appeared to be digits long so I didn't have to adjust for that but there were instances where it was longer than 5 digits long.  For this, I was able to write a simple function that replaced the value with a corrected value if it appeared in a mapping and took the first 5 characters when necessary.

```python
def replace_zipcode(postcode): 
    mapping = {
        'c': '73069'
    }
    if postcode in mapping.keys():
        return mapping[postcode]
    if len(postcode) > 5:
        postcode = postcode[:5]
    return postcode
```

There are additional considerations we should make when thinking about postal codes. Do these zip codes make sense for the area? After importing to SQL I was able to run a query to list all of the zip codes in our data set.


```sql
SELECT tags.value, COUNT(*) as count
FROM (
      SELECT * FROM nodes_tags
      UNION ALL
      SELECT * FROM ways_tags) tags
WHERE tags.key = 'postcode'
GROUP BY tags.value
ORDER BY count DESC;
```
The results are:

| Postal Code 	| Count 	|
|-------------	|------:	|
| 73069 	| 180 	|
| 73071 	| 115 	|
| 73072 	| 93 	|
| 73019 	| 37 	|
| 73026 	| 20 	|
| 73160 	| 8 	|
| 73079 	| 2 	|
| 73060 	| 1 	|
| 73070 	| 1 	|
| 73170 	| 1 	|

It appears that some of the zip codes are from surrounding cities. This is likely caused by the bounding box including some of the surrounding area.

# Data Overview 
The file sizes of the data file I used for this project are as follows: 
* norman.osm - 57.2MB
* norman.db - 30.1MB
* nodes.csv - 21.7MB
* norman-sample.osm - 12.9MB
* ways_nodes.csv - 7.34MB
* ways_tags.csv - 4.49MB
* ways.csv - 1.27MB
* nodes_tags.csv - 636KB

Additional information about the data set and associated SQL queries are as follows:

### Node count: 274,161
```sql
SELECT COUNT(*) FROM nodes;
```
### Way count: 22,400
```sql
SELECT COUNT(*) FROM ways;
```

### Unique Users: 247
```sql
SELECT COUNT(DISTINCT(uid))
FROM (SELECT uid FROM nodes
      UNION ALL
      SELECT uid FROM ways) agg;
```

### Number of McDonalds Restaurants: 3
```sql
SELECT COUNT(id)
FROM nodes_tags 
WHERE value = 'McDonald''s';
```

### Parks: 100
```sql
SELECT COUNT(id)
FROM ways_tags
WHERE key = 'leisure' AND value = 'park';
```

### What is the most common shop type:
```sql
SELECT value, count(*)
FROM nodes_tags
WHERE key = 'shop'
GROUP BY value
ORDER BY count(*) DESC
LIMIT 10;
```
| Shop Type        	| Count 	|
|------------------	|------:	|
| convenience      	|     8 	|
| supermarket      	|     7 	|
| mobile_phone     	|     3 	|
| alcohol          	|     2 	|
| bicycle          	|     2 	|
| books            	|     2 	|
| clothes          	|     2 	|
| department_store 	|     2 	|
| doityourself     	|     2 	|
| fabric           	|     2 	|

It appears we have more convenience stores and supermarkets than anything else. I imagine that many records are missing from this list though as I know we have more department stores and alcohol stores than listed by a large margin. 

### How many restaurants have websites listed: 

```sql
SELECT sitestatus, count(*) as count 
FROM (
    SELECT CASE 
             WHEN website.id IS NULL THEN 'True'
             ELSE 'False'
           END AS sitestatus
    FROM (
          SELECT id 
          FROM nodes_tags
          WHERE key = 'amenity' AND value = 'restaurant'
         ) restaurants
    LEFT JOIN (
          SELECT id 
          FROM nodes_tags
          WHERE key = 'website' ) website ON restaurants.id = website.id
    ) aggregate
GROUP BY sitestatus;

```
| Website | Count |
|---------|-------|
|True     |43     |
|False    |17     |

It looks like the majority of restaurants in this area have a website listed. This is great for people who want to see a menu before going to eat.


### Additional Thoughts
There is additional cleanup that can be done it seems. When I queried the cities in this data I got some bad data back.

```sql
SELECT value, COUNT(*) as count
FROM 
    (SELECT * from nodes_tags UNION ALL SELECT * FROM ways_tags) agg 
WHERE key = 'city' 
GROUP BY value 
ORDER BY count DESC;
```

| City           | Count |
|----------------|-------|
|Norman          |453    |
|Moore           |11     |
|Oklahoma City   |2      |
|"Moore "        |1      |

It looks like I could add a trim function to removing trailing spaces from the city to eliminate this problem. This would give cleaner data and grouping by cities would be more consistent. 


I also question some of the categorization used in this data. There is a separate entry for fast_food and restaurant. I would think it would make more sense to add FastFood as a separate tag for maybe dining type (fast_food, casual, formal, etc.). If that were the case then the top 5 amenities in Norman would look a little different. 

Currently it looks like this:
```sql
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 5;
```
| Amenity        | Count |
|----------------|-------|
|bicycle_parking |298    |
|place_of_worship|77     |
|restaurant      |60     |
|fast_food       |34     |
|parking         |32     |

But if we combine restaurant and fast food
```sql
SELECT value_modified, COUNT(*) as num
FROM (
      SELECT CASE WHEN value = 'fast_food' THEN 'restaurant' ELSE value END AS value_modified
      FROM nodes_tags
      WHERE key='amenity'
     )
GROUP BY value_modified
ORDER BY num DESC
LIMIT 5;
```

| Amenity        | Count |
|----------------|-------|
|bicycle_parking |298    |
|restaurant      |94     |
|place_of_worship|77     |
|parking         |32     |
|fuel            |24     |



# Conclusion
It appears that the data in Norman is fairly complete. This surprised me a bit as it is not a very big city. I think I was able to clean up the data fairly well for this project. I think if there was a way to add additional clarification tags to specific amenities we could get a more granular look at the data. 

### Appendix A - Output from data cleanup
```
Fixed Zipcode: 73072  => 73072
Fixed Street: SW 25th Street => Southwest 25th Street
Fixed Street: South Interstate Drive  => South Interstate Drive
Fixed Street: Riverwalk DR => Riverwalk Drive
Fixed Street: 12th Avenue SE => 12th Avenue Southeast
Fixed Street: S Broadway St => South Broadway Street
Fixed Street: SW 24th St. => Southwest 24th Street
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: David L Boren Blvd => David L Boren Boulevard
Fixed Street: 24th Avenue SW => 24th Avenue Southwest
Fixed Street: 108th Avenue NE => 108th Avenue Northeast
Fixed Street: West Main Steet => West Main Street
Fixed Street: E. Robinson St. => East Robinson Street
Fixed Street: 12th Avenue NE => 12th Avenue Northeast
Fixed Street: Classen Blvd. => Classen Boulevard
Fixed Street: 12th Avenue NE => 12th Avenue Northeast
Fixed Street: 73069 => West Robinson Avenue
Fixed Street: East Hayes Steet => East Hayes Street
Fixed Street: 12th Avenue NE => 12th Avenue Northeast
Fixed Street: 12th Avenue SE => 12th Avenue Southeast
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: Maple => Maple Avenue
Fixed Street: 24th Avenue SE => 24th Avenue Southeast
Fixed Zipcode: c => 73069
Fixed Street: 108th Avenue NE => 108th Avenue Northeast
Fixed Street: SW 19th St => Southwest 19th Street
```